In [1]:
import io
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True)

True

In [10]:
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only


[nltk_data] Downloading package punkt to C:\Users\ANUSHA
[nltk_data]     AGRAWAL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\ANUSHA
[nltk_data]     AGRAWAL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
with open('jaipur.txt','r', encoding='utf8', errors ='ignore') as fin:
    raw = fin.read().lower()
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

In [4]:
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [5]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [6]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [7]:
#CHATBOT
flag=True
print("ROBO: My name is Robo. I will answer your queries about Jaipur. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            greet = greeting(user_response)
            if(greet!=None):
                print("ROBO: "+greet)
            else:
                reply = response(user_response)
                print("ROBO: ",end="")
                print(reply)
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")  
    print()

ROBO: My name is Robo. I will answer your queries about Jaipur. If you want to exit, type Bye!
hello
ROBO: hi there

where is jaipur
ROBO: jaipur is the capital of rajasthan.

places to visit
ROBO: the best time to visit jaipur is between the months of october to march, with occasional showers bathing the city’s streets accompanied with snug weather.

souvenirs
ROBO: you will plenty of souvenirs to take back home with you from the bustling bazaars of jaipur, be it handmade items or old artefacts.

bye
ROBO: Bye! take care..



In [ ]:
from flask import Flask, render_template, request  
#Flask:Used to create a server gateway
#render_template: Used to import the html page and display it in the basic route
#request: Used to get user-input from the website using 'request.args.get()'

#Creating an instance of the class
app = Flask(__name__)     

#Displaying the webpage on the basic route
@app.route("/")
def basic():
    return render_template("chatbot.html")

#Function on the get server to work on the user-input and return the bots reply
@app.route("/get")
def get_bot_response():
    user_response = request.args.get("msg")   
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
            return str("You are welcome..")
        else:
            greet = greeting(user_response)
            if(greet!=None):
                print("ROBO: "+greet)
                return greet
            else:
                reply = response(user_response)
                print("ROBO: ",end="")
                print(reply)
                sent_tokens.remove(user_response)
                return reply
    else:
        flag=False
        print("ROBO: Bye! take care..")  
        return str("Bye! take care..")


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Dec/2021 00:40:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2021 00:40:16] "GET /get?msg=hello HTTP/1.1" 200 -


ROBO: I am glad! You are talking to me


127.0.0.1 - - [08/Dec/2021 00:40:24] "GET /get?msg=where%20is%20jaipur%3F HTTP/1.1" 200 -


ROBO: jaipur is the capital of rajasthan.


127.0.0.1 - - [08/Dec/2021 00:40:42] "GET /get?msg=best%20time%20to%20visit%20jaipur HTTP/1.1" 200 -


ROBO: the best time to visit jaipur is between the months of october to march, with occasional showers bathing the city’s streets accompanied with snug weather.


127.0.0.1 - - [08/Dec/2021 00:40:46] "GET /get?msg=bye HTTP/1.1" 200 -


ROBO: Bye! take care..
